Load graphml file and transform into h5 file using pandana

In [1]:
import networkx as nx

G_temp=nx.read_graphml("./networks/gipuzkoa_drive.graphml")

In [2]:
import pandas as pd

node_df=pd.DataFrame.from_dict(dict(G_temp.nodes(data=True)), orient="index")
edge_df=nx.to_pandas_edgelist(G_temp)

In [3]:
print(node_df.head())

                 y           x street_count highway  ref
448001  43.1795625  -2.4899828            3     NaN  NaN
448006  43.1826552  -2.4814224            4     NaN  NaN
451289  43.1927813  -2.4381722            3     NaN  NaN
469503   43.191805  -2.4512039            3     NaN  NaN
469506  43.1917959  -2.4524942            3     NaN  NaN


In [4]:
# rename length to distance
edge_df=edge_df.rename(columns={"length": "distance"})
print(edge_df.head())

   source      target junction bridge      ref           distance  \
0  448001  3934729820      NaN    NaN      NaN            113.604   
1  448001      916343      NaN    NaN  BI-3344             42.152   
2  448006    25433518      NaN    NaN      NaN             77.905   
3  448006   299660335      NaN    NaN      NaN             229.73   
4  451289  5674243805      NaN    NaN    N-634  907.9949999999999   

                                            geometry width tunnel maxspeed  \
0  LINESTRING (-2.4899828 43.1795625, -2.489874 4...   NaN    NaN      NaN   
1  LINESTRING (-2.4899828 43.1795625, -2.4901355 ...   NaN    NaN      NaN   
2  LINESTRING (-2.4814224 43.1826552, -2.4813444 ...   NaN    NaN      NaN   
3  LINESTRING (-2.4814224 43.1826552, -2.480944 4...   NaN    NaN      NaN   
4  LINESTRING (-2.4381722 43.1927813, -2.4384757 ...   NaN    yes       70   

       highway access                  name oneway lanes reversed  \
0     tertiary    NaN      Otaola hiribidea  Fa

In [5]:
# Convert columns to numeric, forcing errors='coerce' to handle non-numeric values
edge_df["distance"] = pd.to_numeric(edge_df["distance"], errors="coerce")

In [8]:
edge_df["maxspeed"] = pd.to_numeric(edge_df["maxspeed"], errors="coerce")
edge_df["maxspeed"] = edge_df.groupby("highway")["maxspeed"].transform(lambda x: x.fillna(x.max()))
print(edge_df.head())

   source      target junction bridge      ref  distance  \
0  448001  3934729820      NaN    NaN      NaN   113.604   
1  448001      916343      NaN    NaN  BI-3344    42.152   
2  448006    25433518      NaN    NaN      NaN    77.905   
3  448006   299660335      NaN    NaN      NaN   229.730   
4  451289  5674243805      NaN    NaN    N-634   907.995   

                                            geometry width tunnel  maxspeed  \
0  LINESTRING (-2.4899828 43.1795625, -2.489874 4...   NaN    NaN      70.0   
1  LINESTRING (-2.4899828 43.1795625, -2.4901355 ...   NaN    NaN      70.0   
2  LINESTRING (-2.4814224 43.1826552, -2.4813444 ...   NaN    NaN      60.0   
3  LINESTRING (-2.4814224 43.1826552, -2.480944 4...   NaN    NaN      70.0   
4  LINESTRING (-2.4381722 43.1927813, -2.4384757 ...   NaN    yes      70.0   

       highway access                  name oneway lanes reversed  \
0     tertiary    NaN      Otaola hiribidea  False     2    False   
1     tertiary    NaN    G

In [9]:
print(edge_df.groupby("highway")["maxspeed"].apply(lambda x: x.isna().all()))

highway
['living_street', 'residential']               False
['living_street', 'secondary']                 False
['living_street', 'unclassified']               True
['motorway_link', 'motorway']                  False
['motorway_link', 'primary']                    True
['motorway_link', 'residential']                True
['primary', 'secondary']                        True
['primary_link', 'primary']                     True
['tertiary', 'motorway_link']                  False
['tertiary', 'residential', 'secondary']        True
['tertiary', 'residential']                    False
['tertiary', 'secondary']                       True
['tertiary', 'tertiary_link']                  False
['tertiary', 'unclassified', 'residential']     True
['tertiary', 'unclassified']                   False
['tertiary_link', 'secondary']                  True
['trunk_link', 'motorway_link']                 True
['unclassified', 'living_street']               True
['unclassified', 'residential']       

In [10]:
default_speeds = {
    "motorway": 120,
    "motorway_link": 90,
    "trunk": 110,
    "trunk_link": 90,
    "primary": 90,
    "primary_link": 80,
    "secondary": 80,
    "secondary_link": 70,
    "tertiary": 60,
    "tertiary_link": 50,
    "residential": 50,
    "living_street": 30,
    "unclassified": 40,
    "busway": 50,  # Assign reasonable values for missing categories
    "escape": 30,
    "ladder": 10,
    "road": 50  # Generic road
}

def assign_speed(row):
    if pd.notna(row["maxspeed"]):
        return row["maxspeed"]
    
    # Convert list-like strings to actual lists (if necessary)
    highway_types = row["highway"]
    if isinstance(highway_types, str):
        highway_types = eval(highway_types) if highway_types.startswith("[") else [highway_types]
    
    # Find the first matching speed limit
    for road_type in highway_types:
        if road_type in default_speeds:
            return default_speeds[road_type]
    
    # Fallback default speed if none match
    return 50  

# Apply the function to fill NaN values
edge_df["maxspeed"] = edge_df.apply(assign_speed, axis=1)

In [11]:
print(edge_df["maxspeed"].isna().sum())  # Should be 0

0


In [12]:
# save edge_df to csv
edge_df.to_csv("./networks/gipuzkoa_drive_edges.csv", index=False)

In [21]:
print(node_df.head())
print(edge_df.head())

                 x           y
448001  -2.4899828  43.1795625
448006  -2.4814224  43.1826552
451277   -2.425101  43.2049133
451289  -2.4381722  43.1927813
469501  -2.4516244  43.1911116
   source      target  distance
0  448001  3934729820   113.604
1  448001    25438648   139.660
2  448001      916343    42.152
3  448006  5949227426    20.781
4  448006     6928916    78.047


In [22]:
import pandana as pdna

node_x=node_df["x"]
node_y=node_df["y"]
edge_from=edge_df["source"]
edge_to=edge_df["target"]
edge_weights=pd.DataFrame(edge_df["distance"])

walk_network = pdna.Network(node_x, node_y, edge_from, edge_to, edge_weights, twoway=True)

print(walk_network)

In [23]:
network_file="./networks/walk_network.h5"
walk_network.save_hdf5(network_file)